In [1]:
from gscripts.general import parsers
import pandas as pd
import os
from IPython.core.display import HTML
import pybedtools
from clipper.src import CLIP_analysis

import numpy as np
import functools

In [17]:
def eplouge(name,count):
    return """#!/bin/bash
#PBS -N {0}
#PBS -l nodes=1:ppn=1
#PBS -o {0}.out
#PBS -e {0}.err
#PBS -V
#PBS -q home-yeo
#PBS -W group_list=yeo-group
#PBS -t 1-{1}
#PBS -l walltime=8:00:00
cd /home/gpratt/projects/encode/analysis/peak_reanalysis_v12/
echo "hello, starting"
""".format(name, count)

prolouge = "eval ${cmd[$PBS_ARRAYID]}"

In [41]:
def format_clip_analysis(bed_file, bam_file):
    path, ext = os.path.splitext(os.path.basename(bed_file))
    metrics = path + ".metrics"
    return "clip_analysis --clusters {} -s hg19 --bam {} --AS_Structure /projects/ps-yeolab/genomes/hg19/hg19data4 --genome_location /projects/ps-yeolab/genomes/hg19/chromosomes/all.fa --phastcons_location /projects/ps-yeolab/genomes/hg19/hg19_phastcons.bw --nrand 3 --runPhast --metrics {} --gff_db /projects/ps-yeolab/genomes/hg19/gencode.v17.annotation.gtf.db".format(bed_file, bam_file, metrics)

def l2fc_filter(peak_file, l2fc= 3, p_val= 2):
    bedtool = pybedtools.BedTool(peak_file)

    bedtool = bedtool.filter(lambda interval: (float(interval.score) > l2fc) and (float(interval.name) > p_val))
    bedtool = bedtool.each(move_name)
    bedtool = bedtool.saveas(os.path.join("/home/gpratt/projects/encode/analysis/encode_v9_secondary_processing", 
                                         os.path.basename(peak_file)) + ".filtered.bed")
    return bedtool.fn

def move_name(interval):
    interval.name = interval[7]
    return pybedtools.create_interval_from_list(interval[:6])
    return interval

#Run l2f_1 analysis

In [6]:
df = pd.read_table("/home/elvannostrand/data/clip/CLIPseq_analysis/scripts/l2f_1_filelist", 
              header=False, names=['encode_id', 
                                   'rbp', 
                                   'cell_type', 
                                   'bam_file_1', 
                                   'bam_file_2', 
                                   'input_file', 
                                   'peaks_file_1', 
                                   'peaks_file_2'])


#tidy the data up
result = []
for index, line in df.iterrows():
    result.append({'encode_id': line.encode_id,
                   'rep': 1,
                   'rbp': line.rbp, 
                   'cell_type': line.cell_type, 
                   'bam_file': line.bam_file_1, 
                   'input_file': line.input_file, 
                   'peaks_file': line.peaks_file_1, })
    result.append({'encode_id': line.encode_id,
                       'rep': 2,
                       'rbp': line.rbp, 
                       'cell_type': line.cell_type, 
                       'bam_file': line.bam_file_2, 
                       'input_file': line.input_file, 
                       'peaks_file': line.peaks_file_2, })

df = pd.DataFrame(result)

In [7]:
total = 0 
result = []
for x, row in df.iterrows():
    total += 1
    result.append('cmd[{}]="{}"'.format(total, format_clip_analysis(row.peaks_file, row.bam_file)))
    
with open(os.path.join("/home/gpratt/projects/encode/scripts", "encode_l2f_1_analysis.sh"), 'w') as out_file:
    out_file.write(eplouge("encode_l2f_4", total))
    for line in result:
        out_file.write(line + "\n\n")
    out_file.write(prolouge + "\n")

#Run l2f_4 analysis

In [8]:
df = pd.read_table("/home/gpratt/projects/encode/scripts/l2f_4_filelist",
    #"/home/elvannostrand/data/clip/CLIPseq_analysis/scripts/l2f_4_filelist", 
              header=False, names=['encode_id', 
                                   'rbp', 
                                   'cell_type', 
                                   'bam_file_1', 
                                   'bam_file_2', 
                                   'input_file', 
                                   'peaks_file_1', 
                                   'peaks_file_2'])


#tidy the data up
result = []
for index, line in df.iterrows():
    result.append({'encode_id': line.encode_id,
                   'rep': 1,
                   'rbp': line.rbp, 
                   'cell_type': line.cell_type, 
                   'bam_file': line.bam_file_1, 
                   'input_file': line.input_file, 
                   'peaks_file': line.peaks_file_1, })
    result.append({'encode_id': line.encode_id,
                       'rep': 2,
                       'rbp': line.rbp, 
                       'cell_type': line.cell_type, 
                       'bam_file': line.bam_file_2, 
                       'input_file': line.input_file, 
                       'peaks_file': line.peaks_file_2, })

df = pd.DataFrame(result)

total = 0 
result = []
for x, row in df.iterrows():
    total += 1
    result.append('cmd[{}]="{}"'.format(total, format_clip_analysis(row.peaks_file, row.bam_file)))
    
with open(os.path.join("/home/gpratt/projects/encode/scripts", "encode_l2f_4_analysis.sh"), 'w') as out_file:
    out_file.write(eplouge("encode_l2f_4", total))
    for line in result:
        out_file.write(line + "\n\n")
    out_file.write(prolouge + "\n")

#Run Sebas analysis

In [5]:
!head /home/elvannostrand/data/clip/CLIPseq_analysis/SM_v9_20151211/encode_v9_filelist_SM_20151211.txt.l2foldenr_files.txt

SEB1	G3BP	X	/home/gpratt/projects/encode/analysis/encode_v9/G3BP_S_rep1_G3BP1.merged.r2.bam	/home/gpratt/projects/encode/analysis/encode_v9/G3BP_S_rep2_G3BP1.merged.r2.bam	/home/gpratt/projects/encode/analysis/encode_v9/G3BP-S-input_S1_L001_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam	/home/elvannostrand/data/clip/CLIPseq_analysis/SM_v9_20151211/SEB1_01.basedon_SEB1_01.peaks.l2inputnormnew.bed	/home/elvannostrand/data/clip/CLIPseq_analysis/SM_v9_20151211/SEB1_02.basedon_SEB1_02.peaks.l2inputnormnew.bed
SEB2	G3BP	X	/home/gpratt/projects/encode/analysis/encode_v9/G3BP_US_rep1_G3BP1.merged.r2.bam	/home/gpratt/projects/encode/analysis/encode_v9/G3BP_US_rep2_G3BP1.merged.r2.bam	/home/gpratt/projects/encode/analysis/encode_v9/G3BP-US-input_S3_L001_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam	/home/elvannostrand/data/clip/CLIPseq_analysis/SM_v9_20151211/SEB2_01.basedon_SEB2_01.peaks.l2inputnormnew.bed	/home/elvannostrand/data/clip/CLIPseq_analysis/SM_v9_20151211/SE

In [78]:
df = pd.read_table("/home/elvannostrand/data/clip/CLIPseq_analysis/SM_v9_20151211/encode_v9_filelist_SM_20151211.txt.l2foldenr_files.txt",
              header=None, names=['encode_id', 
                                   'rbp', 
                                   'cell_type', 
                                   'bam_file_1', 
                                   'bam_file_2', 
                                   'input_file', 
                                   'peaks_file_1', 
                                   'peaks_file_2'])


#tidy the data up
result = []
for index, line in df.iterrows():
    result.append({'encode_id': line.encode_id,
                   'rep': 1,
                   'rbp': line.rbp, 
                   'cell_type': line.cell_type, 
                   'bam_file': line.bam_file_1, 
                   'input_file': line.input_file, 
                   'peaks_file': line.peaks_file_1, })
    result.append({'encode_id': line.encode_id,
                       'rep': 2,
                       'rbp': line.rbp, 
                       'cell_type': line.cell_type, 
                       'bam_file': line.bam_file_2, 
                       'input_file': line.input_file, 
                       'peaks_file': line.peaks_file_2, })

df = pd.DataFrame(result)
df['annotated_peaks'] = df.peaks_file.apply(lambda x: x + ".compressed.bed.annotated")

In [86]:
df['filtered_peaks'] = df.annotated_peaks.apply(l2fc_filter)

In [83]:
total = 0 
result = []
for x, row in df.iterrows():
    total += 1
    result.append('cmd[{}]="{}"'.format(total, format_clip_analysis(row.filtered_peaks, row.bam_file)))
    
with open(os.path.join("/home/gpratt/projects/encode/scripts", "sebas_analysis.sh"), 'w') as out_file:
    out_file.write(eplouge("sebas", total))
    for line in result:
        out_file.write(line + "\n\n")
    out_file.write(prolouge + "\n")

#Run Analysis for Eric
This may not work anymore, I think I fixed clip analysis to be at the head again

In [7]:
ev42 = format_clip_analysis("/home/elvannostrand/data/clip/CLIPseq_analysis/ALLCLIP_v12_20160112/EV42_01.basedon_EV42_01.peaks.l2inputnormnew.bed.compressed.bed.l10p_5_l2fc_3.bed", 
                     "/projects/ps-yeolab2/encode/analysis/encode_v12/293XT_CLIP_RBFOX2_1120_RBFOX2.merged.r2.bam")

ev43 = format_clip_analysis("/home/elvannostrand/data/clip/CLIPseq_analysis/ALLCLIP_v12_20160112/EV43_01.basedon_EV43_01.peaks.l2inputnormnew.bed.compressed.bed.l10p_5_l2fc_3.bed",
                     "/projects/ps-yeolab2/encode/analysis/encode_v12/293XT_CLIP_RBFOX2_0204_RBFOX2.merged.r2.bam")


#Run Input Normalized Filter for all things

In [42]:
df = pd.read_table("/home/elvannostrand/data/clip/CLIPseq_analysis/KB_v12_20160201/encode_v12_filelist_KB_20160201_nosingle.txt.l2foldenr_files.txt", 
              header=None,
             names=['encode_id', 
                       'rbp', 
                       'cell_type', 
                       'bam_file_1', 
                       'bam_file_2', 
                       'input_file', 
                       'peaks_file_1', 
                       'peaks_file_2'])

#tidy the data up
result = []
for index, line in df.iterrows():
    result.append({'encode_id': line.encode_id,
                   'rep': 1,
                   'rbp': line.rbp, 
                   'cell_type': line.cell_type, 
                   'bam_file': line.bam_file_1, 
                   'input_file': line.input_file, 
                   'peaks_file': line.peaks_file_1, })
    result.append({'encode_id': line.encode_id,
                       'rep': 2,
                       'rbp': line.rbp, 
                       'cell_type': line.cell_type, 
                       'bam_file': line.bam_file_2, 
                       'input_file': line.input_file, 
                       'peaks_file': line.peaks_file_2, })

df = pd.DataFrame(result)
df['annotated_peaks'] = df.peaks_file.apply(lambda x: x + ".compressed.bed.annotated")

In [43]:
df['filtered_peaks'] = df.annotated_peaks.apply(l2fc_filter)

In [55]:
total = 0 
result = []
for x, row in df.iterrows():
    total += 1
    result.append('cmd[{}]="{}"'.format(total, format_clip_analysis(row.filtered_peaks, row.bam_file)))
    
for name, row in df.groupby("encode_id"):
    print name

    rep1, rep2 = row.filtered_peaks

    rep1 = pybedtools.BedTool(rep1)
    rep2 = pybedtools.BedTool(rep2)

    merged_peaks = rep1.intersect(rep2, s=True, u=True).saveas(os.path.join("/home/gpratt/projects/encode/analysis/encode_v9_secondary_processing", 
                                                                            name + ".merged.compressed.bed.annotated.filtered.bed"))
    print len(rep1), len(rep2), len(merged_peaks)
    result.append('cmd[{}]="{}"'.format(total, format_clip_analysis(merged_peaks.fn, row.bam_file.values[0])))
    
with open(os.path.join("/home/gpratt/projects/encode/scripts", "kb_analysis.sh"), 'w') as out_file:
    out_file.write(eplouge("sebas", total))
    for line in result:
        out_file.write(line + "\n\n")
        

    out_file.write(prolouge + "\n")

KB13-14
38 105 12
KB17-18
365 408 117
KB2-3
50 134 25
KB20-21
1526 1276 320
KB4-16
164 283 19
KB6-7
1560 2476 568
KB9-10
1277 651 120
